In [0]:
# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import time
from PIL import Image

from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Input, Dropout
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
df = pd.read_csv('/kaggle/input/plate-zindi/license_plates_detection_train.csv')

# %% [code]
IMAGES_TRAINING = "/kaggle/input/plate-zindi/license_plates_detection_train/license_plates_detection_train/"

# %% [code]
dataset = dict()
dataset["image_name"] = list()
dataset["image_width"] = list()
dataset["image_height"] = list()
dataset["ymin"] = list()
dataset["xmin"] = list()
dataset["ymax"] = list()
dataset["xmax"] = list()   

for index, row in df.iterrows():
    img = Image.open(IMAGES_TRAINING+row['img_id'])
    dataset["image_name"].append(row['img_id'])
    width, height = img.size 
    dataset["image_width"].append(width)
    dataset["image_height"].append(height)
    dataset["ymin"].append(row['ymin'])
    dataset["xmin"].append(row['xmin'])
    dataset["ymax"].append(row['ymax'])
    dataset["xmax"].append(row['xmax'])



# %% [code]
df = pd.DataFrame(dataset)
df.drop(["image_width", "image_height"], axis=1, inplace=True)
ll = df.values.tolist()
for item in ll:
    del item[0]
df['combined']= ll
df

# %% [code]
lucky_test_samples = np.random.randint(0, len(df), 5)
reduced_df = df.drop(lucky_test_samples, axis=0)

# %% [code]
WIDTH = 224
HEIGHT = 224
CHANNEL = 3

def show_img(index):
    image = cv2.imread(IMAGES_TRAINING + df["image_name"].iloc[index])
    width, height = image.shape[0], image.shape[1]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, dsize=(WIDTH, HEIGHT))

    x_scale = ( WIDTH / height)
    y_scale = ( HEIGHT / width )
    tx = int(df["xmin"].iloc[index] * x_scale)
    ty = int(df["ymin"].iloc[index]* y_scale)
    bx = int(df["xmax"].iloc[index]* x_scale)
    by = int(df["ymax"].iloc[index]* y_scale)

    image = cv2.rectangle(image, (tx, ty), (bx, by), (0, 0, 255), 1)
    plt.imshow(image)
    plt.show()

# %% [code]
show_img(1)

# %% [code]
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(
    reduced_df,
    directory=IMAGES_TRAINING,
    x_col="image_name",
    y_col=["combined"],
    target_size=(WIDTH, HEIGHT),
    batch_size=32, 
    class_mode="multi_output",
    subset="training")

validation_generator = datagen.flow_from_dataframe(
    reduced_df,
    directory=IMAGES_TRAINING,
    x_col="image_name",
    y_col=["combined"],
    target_size=(WIDTH, HEIGHT),
    batch_size=32, 
    class_mode="multi_output",
    subset="validation")

# %% [code]
model = Sequential()
model.add(VGG16(weights="imagenet", include_top=False, input_shape=(HEIGHT, WIDTH, CHANNEL)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

model.layers[-6].trainable = False

model.summary()

# %% [code]
STEP_SIZE_TRAIN = int(np.ceil(train_generator.n / train_generator.batch_size))
STEP_SIZE_VAL = int(np.ceil(validation_generator.n / validation_generator.batch_size))

print("Train step size:", STEP_SIZE_TRAIN)
print("Validation step size:", STEP_SIZE_VAL)

train_generator.reset()
validation_generator.reset()

# %% [code]
adam = Adam(lr=0.0005)
model.compile(optimizer=adam, loss="mse")

# %% [code]
history = model.fit_generator(train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VAL,
    epochs=30)

# %% [code]
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()